In [4]:
## FINAL 
import pandas as pd
from pymongo import MongoClient

# Assuming you have pandas DataFrames named as follows:
# df_users: columns - ['user_id', 'skill_score', 'game_id']
# df_games: columns - ['game_id', 'game_name']
# df_location: columns - ['user_id', 'latitude', 'longitude', 'preferred_distance']

# Example DataFrames (Replace with your actual data)
# df_users = pd.DataFrame({
#     'user_id': ['user1', 'user2', 'user3'],
#     'skill_score': [85, 90, 95],
#     'game_id': ['game1', 'game2', 'game3']
# })

# df_games = pd.DataFrame({
#     'game_id': ['game1', 'game2', 'game3'],
#     'game_name': ['Chess', 'Checkers', 'Monopoly']
# })

# df_location = pd.DataFrame({
#     'user_id': ['user1', 'user2', 'user3'],
#     'latitude': [40.7128, 34.0522, 37.7749],
#     'longitude': [-74.0060, -118.2437, -122.4194],
#     'preferred_distance': ['10km', '20km', '30km']
# })

df_users = pd.read_excel(r'dataset\skillscore.xlsx')
df_games = pd.read_excel(r'dataset\game1.xlsx')
df_location = pd.read_excel(r'dataset\location.xlsx')

# Merge DataFrames
df_merged = pd.merge(df_users, df_games, on='GameId')
df_merged = pd.merge(df_merged, df_location, on='UserId')

# Transform data to include game details
df_merged['game_details'] = df_merged.apply(lambda x: {'GameId': x['GameId'], 'Game Name': x['Game Name'], 'Skill Score (out of 10)': x['Skill Score (out of 10)']}, axis=1)

# Group by 'user_id' and aggregate the data
df_final = df_merged.groupby('UserId').agg({
    'game_details': lambda x: list(x),
    'Latitude': lambda x: list(x.unique()),
    'Longitude': lambda x: list(x.unique()),
    'MaxDistance for Connection (in Km)\n*Empty means any distance is fine': lambda x: list(x.unique())
}).reset_index()

# MongoDB Connection
client = MongoClient('mongodb://localhost:27017/')
db = client['Final']  # Replace with your database name
collection = db['final']  # Replace with your collection name

# Insert Data into MongoDB
for data in df_final.to_dict('records'):
    collection.update_one({'_id': data['UserId']}, {'$set': data}, upsert=True)


In [10]:
from pymongo import MongoClient
import math
import numpy as np

def haversine(coord1, coord2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [coord1[0], coord1[1], coord2[0], coord2[1]])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def calculate_average_skill(games):
    skill_scores = [game['Skill Score (out of 10)'] for game in games]
    return np.mean(skill_scores) if skill_scores else 0

def find_friends(user_id, client):
    db = client['Final']
    collection = db['final']

    user = collection.find_one({'_id': user_id})
    if not user:
        return "User not found"

    user_avg_skill = calculate_average_skill(user['game_details'])
    user_location = (user['Latitude'][0], user['Longitude'][0])
    user_pref_distance = user['MaxDistance for Connection (in Km)\n*Empty means any distance is fine'][0]

    potential_friends = []
    for data in collection.find({}):
        if data['_id'] != user_id:
            friend_avg_skill = calculate_average_skill(data['game_details'])
            skill_diff = abs(user_avg_skill - friend_avg_skill)
            distance = haversine(user_location, (data['Latitude'][0], data['Longitude'][0])) if not np.isnan(user_pref_distance) else 0

            if np.isnan(user_pref_distance) or distance <= user_pref_distance:
                potential_friends.append((data['_id'], skill_diff, distance))

    potential_friends.sort(key=lambda x: (x[1], x[2]))  # Sort by skill difference and distance
    return potential_friends[:5]

# MongoDB Connection
client = MongoClient('mongodb://localhost:27017/')

# Example usage
top_5_friends = find_friends(int(input("Enter the user id: ")), client)
print(top_5_friends)


[(20, 0.024999999999999467, 0), (6, 0.128571428571429, 0), (7, 0.22500000000000053, 0), (14, 0.3200000000000003, 0), (18, 0.47499999999999964, 0)]
